In [11]:
import pandas as pd
import numpy as np
import os
import re
pd.set_option('precision',2)

def day(l):
    a = str(l)
    return a[:10]

#Count angry tweets per day
def daycount(filepath):
    df = pd.read_excel(filepath)
    df = df[df['LANGUAGE']=='en'].copy()
    df['bow'] = df['TEXT'].str.lower().str.replace(r'(https?://.+|[^\w#@]+|\d+)+',' ').str.split()
    df = df[df['bow'].isnull()==False].copy()
    df = df[df['TEXT'].str.contains('HDMA')==False].copy()
    df = df[(df['SOURCE'].str.contains('bot', case=False)==False) | (df['SOURCE'].str.contains('tweetbot', case=False)==True)]
    df = df[df['TEXT'].str.contains('FALSE')==False].copy()
    df = df[df['TEXT'].str.contains('TRUE')==False].copy()
    df['Day'] = df['CREATED_AT_LOCAL'].apply(day)
    days = df.pivot_table(index='Day',values='TEXT',aggfunc="count")
    days.columns = ['Angry Tweets']
    city = df['CITY'][0]
    start = days.index.values[0]
    end = days.index.values[-1]
    name = city + ' ' + str(start)[:10] + ' to ' + str(end)[:10] + '.csv'
    days.to_csv(os.path.join(os.path.dirname(filepath),name))

#Count angry tweets per keyword per day    
def dayword(filepath):
    df = pd.read_excel(filepath)
    df = df[df['LANGUAGE']=='en'].copy()
    df['bow'] = df['TEXT'].str.lower().str.replace(r'(https?://.+|[^\w#@]+|\d+)+',' ').str.split()
    df = df[df['bow'].isnull()==False].copy()
    df = df[df['TEXT'].str.contains('HDMA')==False].copy()
    df = df[(df['SOURCE'].str.contains('bot', case=False)==False) | (df['SOURCE'].str.contains('tweetbot', case=False)==True)]
    df = df[df['TEXT'].str.contains('FALSE')==False].copy()
    df = df[df['TEXT'].str.contains('TRUE')==False].copy()
    df['Day'] = df['CREATED_AT_LOCAL'].apply(day)
    df['words'] = df['KEYWORD'].str.replace(r'[^\w\s,]+','').str.split(', ')
    def kwc(l):
        c = []
        for B in l:
            c.extend(B)
        return pd.value_counts(c)
    dw = df.groupby('Day')['words'].apply(kwc).unstack()
    city = df['CITY'][0]
    start = days.index.values[0]
    end = days.index.values[-1]
    name = city + ' ' + str(start)[:10] + ' to ' + str(end)[:10] + ' by word.csv'
    dw.to_csv(os.path.join(os.path.dirname(filepath),name))

#Angry tweets per keyword per day relative to overall keyword percentage    
def daywordratio(filepath):
    df = pd.read_excel(filepath)
    df = df[df['LANGUAGE']=='en'].copy()
    df['bow'] = df['TEXT'].str.lower().str.replace(r'(https?://.+|[^\w#@]+|\d+)+',' ').str.split()
    df = df[df['bow'].isnull()==False].copy()
    df = df[df['TEXT'].str.contains('HDMA')==False].copy()
    df = df[(df['SOURCE'].str.contains('bot', case=False)==False) | (df['SOURCE'].str.contains('tweetbot', case=False)==True)]
    df = df[df['TEXT'].str.contains('FALSE')==False].copy()
    df = df[df['TEXT'].str.contains('TRUE')==False].copy()
    df['Day'] = df['CREATED_AT_LOCAL'].apply(day)
    df['words'] = df['KEYWORD'].str.replace(r'[^\w\s,]+','').str.split(', ')
    def kwc(l):
        c = []
        for B in l:
            c.extend(B)
        return pd.value_counts(c)
    dw = df.groupby('Day')['words'].apply(kwc).unstack()
    city = df['CITY'][0]
    start = days.index.values[0]
    end = days.index.values[-1]
    name = city + ' ' + str(start)[:10] + ' to ' + str(end)[:10] + ' by word'
    dw['Total'] = dw.sum(axis=1)
    dw.loc['Sum'] = dw.sum()
    dwpercent = dw.div(dw['Total'], axis=0)
    dwratio = dwpercent.div(dwpercent.loc['Sum'], axis=1)
    name1 = name + ' ratio.csv'
    name2 = name + ' percent.csv'
    dwratio.to_csv(os.path.join(os.path.dirname(filepath),name1))
    dwpercent.to_csv(os.path.join(os.path.dirname(filepath),name2))

#Count both tweets per day and tweets per keyword    
def day_wordcount(filepath):
    daycount(filepath)
    dayword(filepath)

#Execute any of the above functions on all files in a directory    
def processfolder(directory, function):
    for filename in os.listdir(directory):
        if filename.endswith(".xlsx"):
            function(os.path.join(directory, filename))
            continue
        else:
            continue